# Obtain uniprot id through api

In [1]:
import requests
import numpy as np
import pandas as pd
import json
import os

PATH_ROOT = os.getcwd().replace("\\","/").replace("/notebooks","")


def write_json_to(json_dict,path):
  json_to_write = json.dumps(json_dict)
  write_file = open(path,"w")
  write_file.write(json_to_write)
  write_file.close()

def read_json_from(path):
  with open(path, "r") as read_file:
    return json.load(read_file)

def chunks(lst: [], n: int):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def convert_to_uniprot_id(source_id,source_id_type='ENSEMBL_PRO_ID'):
    url = 'https://www.uniprot.org/uploadlists/'
    params = {
    'from': source_id_type,
    'to': 'ID',
    'format': 'tab',
    'query': source_id
    }
    response = requests.get(url,params)
    for idx,line in enumerate(response.text.splitlines()):
        if idx==0:
            continue
        print(line.split("\t"))

In [30]:
convert_to_uniprot_id('ENSP00000418915')

['ENSP00000418915', 'CDN2A_HUMAN']


This block when run converts ensembl gene id to uniprot id

In [0]:
import numpy as np
import pandas as pd

ensembl_id_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/HI-union-converted.txt"
enseml_id_df = pd.read_csv(ensembl_id_path,delimiter="\t")
enseml_id_df = enseml_id_df.loc[:, ~enseml_id_df.columns.str.contains('^Unnamed')]
enseml_id_df.rename(columns={'Gene_Name_1':'Uniprot_Id_1','Gene_Name_2':'Uniprot_Id_2'}, inplace=True)
converted_1 = ensembl_gene_id_to_uniprot_id(enseml_id_df['Ensembl_Id_1'].tolist())
converted_2 = ensembl_gene_id_to_uniprot_id(enseml_id_df['Ensembl_Id_2'].tolist())
new_df = pd.DataFrame({'Uniprot_Id_1': converted_1,
                   'Uniprot_Id_2': converted_2})
enseml_id_df.update(new_df)
enseml_id_df.head()
enseml_id_df.to_csv("/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/HI-union-converted-fixed.csv",index=False)

This block merge two dataframes into one base on gene name

In [0]:
import numpy as np
import pandas as pd
ensembl_id_converted_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/HI-union-converted.csv"
ensembl_id_converted_df = pd.read_csv(ensembl_id_converted_path)
ensembl_id_converted_df.head()

,Ensembl_Id_1,Ensembl_Id_2,Gene_Name_1,Gene_Name_2
0,ENSG00000000005,ENSG00000061656,TNMD,SPAG4
1,ENSG00000000005,ENSG00000099968,TNMD,BCL2L13
2,ENSG00000000005,ENSG00000104765,TNMD,BNIP3L
3,ENSG00000000005,ENSG00000105383,TNMD,CD33
4,ENSG00000000005,ENSG00000114455,TNMD,HHLA2


In [0]:
gene_name_to_uniprot_id_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/gene-name-to-uniprot-fixed.csv"
gene_name_to_uniprot_id_df = pd.read_csv(gene_name_to_uniprot_id_path)
gene_name_to_uniprot_id_df.head()

,Uniprot_id,Gene_Name
0,Q9H2S6,TNMD
1,A0A0S2Z4U1,DPM1
2,A0A0S2Z4Y5,DPM1
3,H0Y368,DPM1
4,O60762,DPM1


In [0]:
combined_df = pd.merge(ensembl_id_converted_df,gene_name_to_uniprot_id_df,'left',left_on='Gene_Name_1',right_on='Gene_Name')
combined_df.rename(columns={"Uniprot_id":"Uniprot_id_1"},inplace=True)
combined_df.drop(columns='Gene_Name',inplace=True)
combined_df = pd.merge(combined_df,gene_name_to_uniprot_id_df,'left',left_on='Gene_Name_2',right_on='Gene_Name')
combined_df.rename(columns={"Uniprot_id":"Uniprot_id_2"},inplace=True)
combined_df.drop(columns='Gene_Name',inplace=True)

,Ensembl_Id_1,Ensembl_Id_2,Gene_Name_1,Gene_Name_2,Uniprot_id_1,Uniprot_id_2
0,ENSG00000000005,ENSG00000061656,TNMD,SPAG4,Q9H2S6,C9JJZ6
1,ENSG00000000005,ENSG00000061656,TNMD,SPAG4,Q9H2S6,Q5JX49
2,ENSG00000000005,ENSG00000061656,TNMD,SPAG4,Q9H2S6,Q9NPE6
3,ENSG00000000005,ENSG00000099968,TNMD,BCL2L13,Q9H2S6,A0A087WTL4
4,ENSG00000000005,ENSG00000099968,TNMD,BCL2L13,Q9H2S6,A0A087WW80


In [0]:
# combined_df.drop(columns=['Ensembl_Id_1','Ensembl_Id_2','Gene_Name_1','Gene_Name_2'],inplace=True)
# combined_df.head()
# combined_df.to_csv("/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/uniprot-interaction.csv",index=False)

In [0]:
uniprot_interaction_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/uniprot-interaction.csv"
uniprot_interaction_df = pd.read_csv(uniprot_interaction_path)
uniprot_interaction_df.head()

,Uniprot_id_1,Uniprot_id_2
0,Q9H2S6,C9JJZ6
1,Q9H2S6,Q5JX49
2,Q9H2S6,Q9NPE6
3,Q9H2S6,A0A087WTL4
4,Q9H2S6,A0A087WW80


The following code process gene-name-to-uniprot-fixed.csv by discarding row with empty entries, and adding rows with more than one elements in each row

In [0]:
import numpy as np
import pandas as pd

huri_union_converted_fixed_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/HI-union-converted-fixed.csv"
huri_union_converted_fixed_df = pd.read_csv(huri_union_converted_fixed_path)
huri_union_converted_fixed_df = huri_union_converted_fixed_df[['Uniprot_Id_1','Uniprot_Id_2']]
print(huri_union_converted_fixed_df)

                   Uniprot_Id_1              Uniprot_Id_2
0                        Q9H2S6                    Q9NPE6
1                        Q9H2S6                    Q9BXK5
2                        Q9H2S6                    O60238
3                        Q9H2S6                    P20138
4                        Q9H2S6                    Q9UM44
...                         ...                       ...
64001                    B2RXH8                    B2RXH8
64002                    Q8NHW4                    Q6IN84
64003  ['P02489', 'A0A140G945']  ['P02489', 'A0A140G945']
64004                    Q9UI36                    Q9UI36
64005                    P43243                    P43243

[64006 rows x 2 columns]


In [0]:
import math

rows_to_throw_out = []
rows_to_add = []
prevent_repeat_set = set()
for index, row in huri_union_converted_fixed_df.iterrows():
  item1 = str(row[0]).strip("'")
  item2 = str(row[1]).strip("'")
  if (item1 == 'nan' or item2 == 'nan') or item1==item2 or item1+item2 in prevent_repeat_set:
    rows_to_throw_out.append(index)
  elif '[' in item1 or '[' in item2:
    rows_to_throw_out.append(index)
    list1 = []
    list2 = []
    if not '[' in item1:
      list1.append(item1)
    else:
      list1 = item1.strip('][').strip("'").split(', ') 
    if not '[' in item2:
      list2.append(item2)
    else:
      list2 = item2.strip('][').strip("'").split(', ') 
    for id1 in list1:
      for id2 in list2:
        id1_strip = id1.strip("'")
        id2_strip = id2.strip("'")
        if not id1_strip + id2_strip in prevent_repeat_set and id1_strip!=id2_strip:
          rows_to_add.append([id1_strip,id2_strip])
          prevent_repeat_set.add(id1_strip+id2_strip)
          prevent_repeat_set.add(id2_strip+id1_strip)
  else:
    prevent_repeat_set.add(item1+item2)
    prevent_repeat_set.add(item2+item1)

In [0]:
print(len(rows_to_throw_out))
print(len(rows_to_add))

4108
1994


In [0]:
huri_union_converted_fixed_df_dropped = huri_union_converted_fixed_df.drop(huri_union_converted_fixed_df.index[rows_to_throw_out])
print(huri_union_converted_fixed_df_dropped)

      Uniprot_Id_1 Uniprot_Id_2
0           Q9H2S6       Q9NPE6
1           Q9H2S6       Q9BXK5
2           Q9H2S6       O60238
3           Q9H2S6       P20138
4           Q9H2S6       Q9UM44
...            ...          ...
63994       P60409       Q9UGY1
63995       P60409       Q9BSG1
63996       P60409       Q9BS34
64000       P13236       Q6IN84
64002       Q8NHW4       Q6IN84

[59898 rows x 2 columns]


In [0]:
temp_df = pd.DataFrame(rows_to_add,columns=['Uniprot_Id_1','Uniprot_Id_2'])
print(temp_df)

     Uniprot_Id_1 Uniprot_Id_2
0          Q14847       P86481
1          Q14847       P86496
2          Q14847       P86479
3          Q14847       P86480
4          Q14847       P86478
...           ...          ...
1989       Q7Z3S9       Q96JP2
1990       P0DPK4       P60409
1991       Q7Z3S9       P60409
1992       Q5HYN5       P0DMU9
1993       Q5HYN5       P0DMU8

[1994 rows x 2 columns]


In [0]:
final_df = huri_union_converted_fixed_df_dropped.append(temp_df)
print(final_df)

     Uniprot_Id_1 Uniprot_Id_2
0          Q9H2S6       Q9NPE6
1          Q9H2S6       Q9BXK5
2          Q9H2S6       O60238
3          Q9H2S6       P20138
4          Q9H2S6       Q9UM44
...           ...          ...
1989       Q7Z3S9       Q96JP2
1990       P0DPK4       P60409
1991       Q7Z3S9       P60409
1992       Q5HYN5       P0DMU9
1993       Q5HYN5       P0DMU8

[61892 rows x 2 columns]


In [0]:
final_df.to_csv("/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/HI-union-processed.csv",index=False)

# Obtain uniprot id through direct download from uniprot then merging dataframe

In [3]:
import numpy as np
import pandas as pd

pQTL_ensp_to_uniprot_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/pQTL_ensp_to_uniprot.csv"
pQTL_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/pQTL.csv"

pQTL_mapping_df = pd.read_csv(pQTL_ensp_to_uniprot_path)
pQTL_df = pd.read_csv(pQTL_path)
print(pQTL_mapping_df.head)
print(pQTL_df.head)


<bound method NDFrame.head of           Column1          Column2
0      Uniprot_Id       Ensembl_Id
1          H0Y368  ENSG00000000419
2          O60762  ENSG00000000419
3          Q5QPJ9  ENSG00000000419
4          Q5QPK2  ENSG00000000419
...           ...              ...
21467      P63151  ENSG00000221914
21468      M0R1M6  ENSG00000221983
21469      M0R1V7  ENSG00000221983
21470      M0R2S1  ENSG00000221983
21471      P62987  ENSG00000221983

[21472 rows x 2 columns]>
<bound method NDFrame.head of                  ENSG   GM18486   GM18498  ...    txStart      txEnd    chr
0     ENSG00000000419 -0.152939  0.112534  ...   48984810   49008499  chr20
1     ENSG00000000457  0.791652       NaN  ...  168088838  168129670   chr1
2     ENSG00000000938 -2.005159 -1.938036  ...   27811389   27834314   chr1
3     ENSG00000001084 -2.120697 -1.606023  ...   53470097   53517790   chr6
4     ENSG00000001630  0.374041 -0.734608  ...   91579408   91601946   chr7
...               ...       ...      

In [5]:
print(pQTL_mapping_df.columns)

Index(['Column1', 'Column2'], dtype='object')


In [11]:
pQTL_combined_df = pd.merge(pQTL_df,pQTL_mapping_df,'left',left_on='ENSG',right_on='Column2')
pQTL_combined_df = pQTL_combined_df.rename(columns={'Column1': 'Uniprot_Id'})
pQTL_combined_df = pQTL_combined_df.drop(columns=['Column2'])
cols = pQTL_combined_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
pQTL_combined_df = pQTL_combined_df[cols]
pQTL_combined_df = pQTL_combined_df[pQTL_combined_df['Uniprot_Id'].notna()]
print(pQTL_combined_df)

      Uniprot_Id             ENSG   GM18486  ...    txStart      txEnd    chr
0         H0Y368  ENSG00000000419 -0.152939  ...   48984810   49008499  chr20
1         O60762  ENSG00000000419 -0.152939  ...   48984810   49008499  chr20
2         Q5QPJ9  ENSG00000000419 -0.152939  ...   48984810   49008499  chr20
3         Q5QPK2  ENSG00000000419 -0.152939  ...   48984810   49008499  chr20
4         Q8IZE3  ENSG00000000457  0.791652  ...  168088838  168129670   chr1
...          ...              ...       ...  ...        ...        ...    ...
21504     P63151  ENSG00000221914  0.138373  ...   26204950   26284562   chr8
21505     M0R1M6  ENSG00000221983  0.154847  ...   18545110   18546960  chr19
21506     M0R1V7  ENSG00000221983  0.154847  ...   18545110   18546960  chr19
21507     M0R2S1  ENSG00000221983  0.154847  ...   18545110   18546960  chr19
21508     P62987  ENSG00000221983  0.154847  ...   18545110   18546960  chr19

[21471 rows x 67 columns]


In [0]:
csv_out_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/data_sources/pQTL_combined.csv"
pQTL_combined_df.to_csv(csv_out_path,index=False)

In [54]:
huge_json

{'P84085': {'Q07866': {'neighborhood': 0,
   'neighborhood_transferred': 0,
   'fusion': 0,
   'cooccurence': 0,
   'homology': 0,
   'coexpression': 0,
   'coexpression_transferred': 55,
   'experiments': 0,
   'experiments_transferred': 78,
   'database': 900,
   'database_transferred': 0,
   'textmining': 0,
   'textmining_transferred': 0,
   'combined_score': 905},
  'Q9ULH1': {'neighborhood': 0,
   'neighborhood_transferred': 0,
   'fusion': 0,
   'cooccurence': 0,
   'homology': 0,
   'coexpression': 0,
   'coexpression_transferred': 0,
   'experiments': 0,
   'experiments_transferred': 172,
   'database': 0,
   'database_transferred': 0,
   'textmining': 716,
   'textmining_transferred': 452,
   'combined_score': 859},
  'O95398': {'neighborhood': 0,
   'neighborhood_transferred': 0,
   'fusion': 0,
   'cooccurence': 0,
   'homology': 0,
   'coexpression': 0,
   'coexpression_transferred': 0,
   'experiments': 0,
   'experiments_transferred': 180,
   'database': 0,
   'database_

In [51]:
colnames

['neighborhood',
 'neighborhood_transferred',
 'fusion',
 'cooccurence',
 'homology',
 'coexpression',
 'coexpression_transferred',
 'experiments',
 'experiments_transferred',
 'database',
 'database_transferred',
 'textmining',
 'textmining_transferred',
 'combined_score']